In [679]:
import numpy as np

1. Gaussian Naive Bayes Classifier

$$
P(x_i|y)=\frac{1}{\sqrt{2\pi\sigma_y^2}}\exp \Big(-\frac{(x_i-\mu_y)^2}{2\sigma_y^2}\Big)
$$

In [680]:
class GaussianNB:
    
    def __init__(self, priors = None, minimum_var = 1e-09):
        """
        GaussianNB Classifier.

        Parameters
        ------------------
        priors: {array-like} 
        
            set priors for each class if it is known, unless set it as None
        
        minimum_var: float
            
            To avoid 0 variance.

        Attributes
        ------------------        
        """
        self._class_prior = priors
        self._class_count = None
        self._classes = None
        self._var =None
        self._mean = None
        self.minimum_var = minimum_var
    
    
    def fit(self, X_train, y_train):
        """
        Training model
        
        Parameters
        ------------------
        X_train : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y_train : array-like of shape (n_samples,)
            Target values.
   
        """
        
        # Check datatype 
        if isinstance(X_train,np.ndarray) and isinstance(y_train,np.ndarray):
            pass
        else:
            try:
                X_train = np.array(X_train)
                y_train = np.array(y_train)
            except:
                raise TypeError('numpy.array required for input data')
        
        # Sample and feature size
        
        m_sample, n_feature = X_train.shape 
        
        # Find all unique classes
        self._classes = np.unique(y_train)
        self._class_count = len(self._classes)
        
        # Initialize man, var, prior for each class
        if self._class_prior == None: # No priors
            
            self._class_prior = np.zeros(self._class_count,dtype=np.float64)
            self._var = np.zeros((self._class_count,n_feature),dtype=np.float64)
            self._mean = np.zeros((self._class_count,n_feature),dtype=np.float64)

            for idx, cl in enumerate(self._classes):
                X_cl = X_train[y_train==cl]
                self._class_prior[idx] = X_cl.shape[0]/float(m_sample)
                self._mean[idx, :] = X_cl.mean(axis=0) # along column
                self._var[idx, :] = X_cl.var(axis=0) 

        else:
            
            self._var = np.zeros((self._class_count,n_feature),dtype=float)
            self._mean = np.zeros((self._class_count,n_feature),dtype=float)

            for idx, cl in enumerate(self._classes):
                X_cl = X_train[y_train==cl]
                self._mean[idx, :] = X_cl.mean(axis=0) # along column
                self._var[idx, :] = X_cl.var(axis=0) 
        
        self._var[:,:] += self.minimum_var
                
            
    def predict(self, X_test):
        y_pred = [self._predict(x_test) for x_test in X_test]
        return np.array(y_pred)
    
    
    def _predict(self, x_test):
        
        posteriors = []
        
        # Instead of multiply all probabilities, we calculate the sum of log(p)
        
        for idx, cl in enumerate(self._classes):
            
            likelihood = np.sum(np.log(self._pdf(idx,x_test)))
            prior = np.log(self._class_prior[idx])
            
            posterior = likelihood + prior
            
            posteriors.append(posterior)
            
        # return the class with highest posterior
        return self._classes[np.argmax(posteriors)]
        
    
    def _pdf(self, class_idx, x_test):
        
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x_test - mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator/denominator
        
        

2. Multinomial Naive Bayes

Likelihood

$$
P(x_i|y)=\frac{Count(x_i|y)+\alpha}{Count(y)+\alpha|V|}
$$
or
$$
P(x_i|y)=\frac{\sum_{i=1}^{N}I(x_i,y)+\alpha}{\sum_{i=1}^{N}I(y)+\alpha|V|}
$$
where $|V|$ is the number of different categories in $x_i$, and $\alpha$ is a smoothing parameter and $\alpha>0$.

Credit to [Kenzo takahashi](https://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html)

In [681]:

class MultinomialNB:
    
    def __init__(self, alpha = 1.0):
        """
        MultinomialNB Classifier.

        Parameters
        ------------------
        priors: {array-like} 
        
            set priors for each class if it is known, unless set it as None
            
        alpha: float, default = 1.0 
        
            alpha = 1.0: Laplace smoothing
            alpha in (0,1): Lidstone
            alpha = 0: No smoothing
        

        Attributes
        ------------------ 
        """
        self.alpha = alpha
        self._classes = None
        self._class_count = None
        self._classes_groups = None
        self._class_log_prior = None
        self._feature_count = None
        self._feature_log_prob = None

        
    def fit(self, X_train, y_train):
        
        """
        Training model
        
        Parameters
        ------------------
        X_train : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y_train : array-like of shape (n_samples,)
            Target values.
   
        """
        
        # Check datatype 
        if isinstance(X_train,np.ndarray) and isinstance(y_train,np.ndarray):
            pass
        else:
            try:
                X_train = np.array(X_train)
                y_train = np.array(y_train)
            except:
                raise TypeError('numpy.array required for input data')
        
        # Sample and feature size
        
        m_sample, n_feature = X_train.shape  
        
        # Find all unique classes
        self._classes = np.unique(y_train)
        # Group samples by its class 
        self._classes_groups = [[x for x, t in zip(X_train, y_train) if t == c] for c in np.unique(y_train)]
        # Count numner of samples in each class
        self._class_count = [len(x) for x in self._classes_groups] 
        # Calculate the log prior of each class
        self._class_log_prior =  [np.log(len(cl)/float(m_sample)) for cl in self._classes_groups]
        # Count each feature/word for each class
        self._feature_count = np.array([np.array(cl).sum(axis=0) for cl in self._classes_groups], dtype=float) 
        # Add alpha to self._feature_count
        count = self._feature_count + self.alpha
        # Caculate log conditional probability 
        # [np.newaxis]: np can not transpose 1-D array only by .T
        self._feature_log_prob = np.log( count / count.sum(axis=1)[np.newaxis].T)

        
    def predict(self, X_test):
        # Check datatype 
        if isinstance(X_test,np.ndarray):
            pass
        else:
            try:
                X_test = np.array(X_test)
            except:
                raise TypeError('numpy.array required for input data')
        
        predictions = [self._predict(x) for x in X_test]
        return predictions

                
    def _predict(self, x):

        posteriors = [(self._feature_log_prob*x).sum(axis=1) + self._class_log_prior]

        return self._classes[np.argmax(posteriors)]
        

        

3. BernoulliNB

Similar to multinomial naive bayes, but only include binary values, number or boolean.

$$
P(x_i|y)=P(i|y)x_i + (1-P(i|y)(1-x_i))
$$

In [682]:

class BernoulliNB:
    
    def __init__(self, alpha = 1.0, binarize = None):
        """
        BernoulliNB Classifier.

        Parameters
        ------------------
        priors: {array-like} 
        
            set priors for each class if it is known, unless set it as None
            
        alpha: float, default = 1.0 
        
            alpha = 1.0: Laplace smoothing
            alpha in (0,1): Lidstone
            alpha = 0: No smoothing
        
        binarize: float, default = None
        
            The threshold for binarizing of sample feature. If None, 
            input is presumed to already consist of binary vectors.

        Attributes
        ------------------ 
        """
        self.alpha = alpha
        self._classes = None
        self._class_count = None
        self._classes_groups = None
        self._class_log_prior = None
        self._feature_count = None
        self._feature_prob = None
        self.binarize = binarize

        
    def fit(self, X_train, y_train):
        
        """
        Training model
        
        Parameters
        ------------------
        X_train : array-like of shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y_train : array-like of shape (n_samples,)
            Target values.
   
        """
        
        # Check datatype 
        if isinstance(X_train,np.ndarray) and isinstance(y_train,np.ndarray):
            pass
        else:
            try:
                X_train = np.array(X_train)
                y_train = np.array(y_train)
            except:
                raise TypeError('numpy.array required for input data')
                
        # Check if X_train is binary
        if self.binarize == None:
            pass
        else:
            try:
                np.where(X_train>self.binarize, 1, 0)
            except:
                raise TypeError('X requires to be binary')

        
        # Sample and feature size
        m_sample, n_feature = X_train.shape  
        
        # Find all unique classes
        self._classes = np.unique(y_train)
        # Group samples by its class 
        self._classes_groups = [[x for x, t in zip(X_train, y_train) if t == c] for c in np.unique(y_train)]
        # Count numner of samples in each class
        self._class_count = [len(x) for x in self._classes_groups]
        # Calculate the log prior of each class
        self._class_log_prior =  [np.log(len(cl)/float(m_sample)) for cl in self._classes_groups]
        # Count each feature/word for each class
        self._feature_count = np.array([np.array(cl).sum(axis=0) for cl in self._classes_groups], dtype=float) 
        # Caculate conditional log probability 
        # [np.newaxis]: np can not transpose 1-D array only by .T
        self._feature_log_prob = np.log( (self._feature_count + self.alpha)  / (np.array(self._class_count) + 2*self.alpha)[np.newaxis].T)

        
    def predict(self, X_test):
        # Check datatype 
        if isinstance(X_test,np.ndarray):
            pass
        else:
            try:
                X_test = np.array(X_test)
            except:
                raise TypeError('numpy.array required for input data')
        
        predictions = [self._predict(x) for x in X_test]
        return predictions

                
    def _predict(self, x):

        posteriors = [(self._feature_log_prob*x - self._feature_log_prob*(1-x)).sum(axis=1) + self._class_log_prior]
        return self._classes[np.argmax(posteriors)]
    
    

In [689]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB as GNB
import matplotlib.pyplot as plt

# from naivebayes import NaiveBayes


def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

X, y = datasets.make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


nb = GaussianNB()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

gnb = GNB()
gnb.fit(X_train, y_train)
predictions2 = gnb.predict(X_test)

print("Current Model accuracy {} vs sklearn Model {}".format (accuracy(y_test, predictions),accuracy(y_test, predictions2)))


Current Model accuracy 0.965 vs sklearn Model 0.965
